# Ejemplo de WebScraping con Python

## Obtener S&P Merval que es el principal índice del Mercado de Valores de Buenos Aires

In [ ]:
# Importamos las librerías Python que utilizaremos
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime

In [ ]:
# Indicamos la ruta de la web que deseamos acceder
url_page = 'https://bolsar.info'

In [ ]:
# Y ahora haremos el request a esa ruta y procesaremos el HTML mediante un objeto de tipo BeautifulSoap. 

#page = requests.get(url_page).text 
page = requests.get(url_page, verify=False)

# Verificar si la solicitud fue exitosa
if page.status_code == 200:
    soup = BeautifulSoup(page.text, "html.parser")
    
    # Por ejemplo, puedes imprimir el título de la página:
    print(soup.title.text)

    #print(soup)
else:
    print("No se pudo acceder a la página web.")

Bien, ahora nos toca pensar la estrategia para acceder al valor. En nuestro caso nos interesa primero acceder a la tabla, y de allí a sus celdas. Por suerte la tabla tiene un id único!

In [ ]:
# Buscar el elemento <table> con el atributo 'id' igual a 'tbAlzasBajasSinCambio'
tabla = soup.find('table', attrs={'id':'tbAlzasBajasSinCambio'})

# Verificar si se encontró la tabla
if tabla:
    # Realizar acciones en la tabla encontrada
    print(tabla)
else:
    print("La tabla no fue encontrada en la página.")

Nos devuelve el `<tbody>` vacío y la información que deseamos está dentro de este `<tbody>`generado dinámicamente a través de JavaScript, `BeautifulSoup`, por sí solo, no puede capturar esa información, ya que se trata de una técnica conocida como carga dinámica de contenido.

Para capturar datos generados dinámicamente a través de JavaScript, es posible que necesitmos una herramienta adicional como `Selenium`, que puede controlar un navegador web y ejecutar JavaScript. Aquí hay un ejemplo de cómo usar Selenium con Chrome WebDriver para obtener la página y luego extraer la información de la tabla:

Primero, debes instalar `Selenium` si aún no lo hemos hecho:

    $ pip install selenium


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Opciones para el navegador Chrome
chrome_options = Options()
chrome_options.add_argument('--headless')  # Ejecutar Chrome en modo sin cabeza (sin interfaz gráfica)

# Inicializar el controlador de Chrome
driver = webdriver.Chrome(options=chrome_options)

# Acceder a la página web con Selenium
driver.get(url_page)

# Esperar a que la página se cargue completamente (puedes ajustar este tiempo según sea necesario)
driver.implicitly_wait(10)

# Obtener el contenido HTML de la página después de que JavaScript haya cargado los datos
page_source = driver.page_source

# Cerrar el navegador web controlado por Selenium
driver.quit()

# Analizar el contenido HTML con BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Buscar la tabla dentro del HTML cargado dinámicamente
tabla = soup.find('table', id='tbAlzasBajasSinCambio')

# Verificar si se encontró la tabla
if tabla:
    # Realizar acciones en la tabla encontrada
    print(tabla)
else:
    print("La tabla no fue encontrada en la página.")


In [ ]:
# Obtener todas las filas de la tabla
filas = tabla.find_all('tr')
    
# Recorrer las filas y obtener los valores de las celdas
for fila in filas:
    # Obtener todas las celdas de la fila
    celdas = fila.find_all('td')
        
    # Imprimir los valores de las celdas
    for celda in celdas:
        print(celda.text.strip())  # Utiliza strip() para eliminar espacios en blanco alrededor de los valores
print()  # Agregar una línea en blanco entre las filas

In [ ]:
# Buscar todas las filas de la tabla
filas = tabla.find_all('tr')
    
# Iterar a través de las filas
for fila in filas:
    # Buscar las celdas de la fila
    celdas = fila.find_all('td')
        
    # Verificar si la fila contiene el índice "S&P MERVAL"
    if any("S&P MERVAL" in celda.text for celda in celdas):
        # Si la fila contiene el índice, almacenar los valores de las celdas en variables
        valores_celda = [celda.text.strip() for celda in celdas]
            
        # Imprimir los valores de las celdas
        print(valores_celda)

In [ ]:
# Abrimos el csv con append para que pueda agregar contenidos al final del archivo
with open('bolsa_m.csv', 'a') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow([valores_celda[0], valores_celda[1], valores_celda[2], datetime.now()])